In [1]:
library(ArchR)
library(Seurat)
library(Signac)
library(magrittr)
library(genomation)
library(GenomicRanges)
library(Matrix)
library(ggplot2)
library(dplyr)


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--' ,----`-,__ ___/'  --,-`-===================##========>
       \               '        ##_______ _____ ,--,__,=##,__   ///
        ,    __==    ___,-,__,--'#'  ==='      `-'    | ##,-/
        -,____,---'       \\####\\________________,--\\_##,/
           ___      .______        ______  __    __  .______      
          /   \     |   _ 

Specify file path

In [2]:
gene_gtf_path = "/maps/projects/ralab/data/genome/hg38/gencode.v43.chr_patch_hapl_scaff.annotation.gtf"
abc_genes_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.TSS500bp.bed"
TSS_path = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/241203/scE2G/ENCODE_rE2G/ABC/reference/hg38/CollapsedGeneBounds.hg38.bed"
path.atac_frag = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/scE2G_input_Maya/Jurkat/atac_fragments.tsv.gz"
path.matrix.rna_count = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/processed/scE2G_input_Maya/Jurkat/rna_count_matrix.csv.gz"
path.pairs.E2G = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/software/scE2G_pipeline/250319/scE2G/test/results/Jurkat/Jurkat/Kendall/Pairs.tsv.gz"
dir.output = "/maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/Jurkat/1.Genome_wide_prediction/ArchR/ArchR/"
celltype = "Jurkat"

In [3]:
n.cores = 16
maxDist = 1000000

In [4]:
addArchRThreads(threads = n.cores) 
addArchRGenome("hg38")

Setting default number of Parallel threads to 16.

Setting default genome to Hg38.



Import candidate E-G pairs

In [5]:
pairs.E2G = readGeneric(path.pairs.E2G,
                        header = T,
                        keep.all.metadata = T)

Import RNA matrix

In [6]:
matrix.rna = read.csv(path.matrix.rna_count,
                      row.names = 1,
                      check.names = F)
matrix.rna = Matrix(as.matrix(matrix.rna), sparse = TRUE)
# matrix.rna = matrix.rna[,colnames(matrix.atac)]
matrix.rna = matrix.rna[rowSums(matrix.rna) > 0,]

Map gene names

In [7]:
extract_attributes <- function(gtf_attributes, att_of_interest){
  att <- unlist(strsplit(gtf_attributes, " "))
  if(att_of_interest %in% att){
    return(gsub("\"|;","", att[which(att %in% att_of_interest)+1]))
  } else {
    return(NA)}
}
map_gene_names <- function(rna_matrix, gene_gtf_path, abc_genes_path){
	gene_ref <- fread(gene_gtf_path, header = FALSE, sep = "\t") %>%
		setNames(c("chr","source","type","start","end","score","strand","phase","attributes")) %>%
		dplyr::filter(type == "gene")
	gene_ref$gene_ref_name <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_name"))
	gene_ref$Ensembl_ID <- unlist(lapply(gene_ref$attributes, extract_attributes, "gene_id"))
	gene_ref <- dplyr::select(gene_ref, gene_ref_name, Ensembl_ID) %>%
		mutate(Ensembl_ID = sub("\\.\\d+$", "", Ensembl_ID)) %>% # remove decimal digits 
		distinct()
	
	abc_genes <- fread(abc_genes_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type")) %>%
		dplyr::select(name, Ensembl_ID) %>%
		rename(abc_name = name) %>%
		left_join(gene_ref, by = "Ensembl_ID") %>%
		group_by(Ensembl_ID) %>% # remove cases where multiple genes map to one ensembl ID
		filter(n() == 1) %>%
		ungroup()

	gene_key <- abc_genes$abc_name
	names(gene_key) <- abc_genes$gene_ref_name

	# remove genes not in our gene universe	
	row_sub <- intersect(rownames(rna_matrix), names(gene_key)) # gene ref names
	rna_matrix_filt <- rna_matrix[row_sub,] # still gene ref names
	rownames(rna_matrix_filt) <- gene_key[row_sub] # converted to abc names

	return(rna_matrix_filt)
}

In [8]:
matrix.rna.rename = map_gene_names(matrix.rna,gene_gtf_path, abc_genes_path)

make TSS GRanges

In [9]:
df.tss <- fread(TSS_path, col.names = c("chr", "start", "end", "name", "score", "strand", "Ensembl_ID", "gene_type"))
df.tss = as.data.frame(df.tss)
rownames(df.tss) = df.tss$name

In [10]:
df.tss[df.tss$strand == "+","end"] = df.tss[df.tss$strand == "+","start"]
df.tss[df.tss$strand == "-","start"] = df.tss[df.tss$strand == "-","end"]

In [11]:
df.tss = df.tss[unname(rownames(matrix.rna.rename)),]

In [12]:
gr.tss <- makeGRangesFromDataFrame(
  df.tss,
  seqnames.field = "chr",
  start.field = "start",
  end.field = "end",
  strand.field = "strand",    
  keep.extra.columns = TRUE   
)

In [13]:
gr.tss

GRanges object with 17571 ranges and 4 metadata columns:
            seqnames    ranges strand |        name     score      Ensembl_ID
               <Rle> <IRanges>  <Rle> | <character> <integer>     <character>
      OR4F5     chr1     69090      + |       OR4F5         0 ENSG00000186092
     FAM87B     chr1    817370      + |      FAM87B         0 ENSG00000177757
  LINC01128     chr1    827590      + |   LINC01128         0 ENSG00000228794
     FAM41C     chr1    876802      - |      FAM41C         0 ENSG00000230368
     SAMD11     chr1    925740      + |      SAMD11         0 ENSG00000187634
        ...      ...       ...    ... .         ...       ...             ...
     NLGN4Y     chrY  14523745      + |      NLGN4Y         0 ENSG00000165246
     TTTY14     chrY  19077547      - |      TTTY14         0 ENSG00000176728
      KDM5D     chrY  19744939      - |       KDM5D         0 ENSG00000012817
     TTTY10     chrY  20519228      - |      TTTY10         0 ENSG00000229236
     EI

Create an ArchRProject

In [14]:
ArrowFiles <- createArrowFiles(
  inputFiles = c(path.atac_frag),
  sampleNames = c(celltype),
  validBarcodes = list(Jurkat = colnames(matrix.rna.rename)),
  addTileMat = T,
  addGeneScoreMat = F,
  force = T
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

ArchR logging to : ArchRLogs/ArchR-createArrows-6788d2a6a5d07-Date-2025-03-22_Time-16-52-34.894808.log
If there is an issue, please report to github with logFile!

Cleaning Temporary Files

2025-03-22 16:52:35.292951 : Batch Execution w/ safelapply!, 0 mins elapsed.

(Jurkat : 1 of 1) Determining Arrow Method to use!

2025-03-22 16:52:35.34847 : (Jurkat : 1 of 1) Reading In Fragments from inputFiles (readMethod = tabix), 0.001 mins elapsed.

2025-03-22 16:52:35.356103 : (Jurkat : 1 of 1) Tabix Bed To Temporary File, 0.001 mins elapsed.

2025-03-22 16:53:41.194705 : (Jurkat : 1 of 1) Successful creation of Temporary File, 1.098 mins elapsed.

2025-03-22 16:53:41.206138 : (Jurkat : 1 of 1) Creating ArrowFile From Temporary File, 1.099 mins elapsed.

2025-03-22 16:54:07.606263 : (Jurkat : 1 of 1) Successful creation of Arrow File, 1.539 mins elapsed.

2025-03-22 16:54:21.79745 : Continuin

In [15]:
ArrowFiles

[1] "Jurkat.arrow"

In [16]:
dir.create(paste(dir.output,"ArchR",sep="/"),recursive = T)

In [17]:
obj.ArchR <- ArchRProject(
  ArrowFiles = ArrowFiles, 
  outputDirectory = paste(dir.output,"ArchR",sep="/"),
  copyArrows = TRUE 
)

Using GeneAnnotation set by addArchRGenome(Hg38)!

Using GeneAnnotation set by addArchRGenome(Hg38)!

Validating Arrows...

Getting SampleNames...

1 


Copying ArrowFiles to Ouptut Directory! If you want to save disk space set copyArrows = FALSE

1 


Getting Cell Metadata...

1 


Merging Cell Metadata...

Initializing ArchRProject...


                                                   / |
                                                 /    \
            .                                  /      |.
            \\\                              /        |.
              \\\                          /           `|.
                \\\                      /              |.
                  \                    /                |\
                  \\#####\           /                  ||
                ==###########>      /                   ||
                 \\##==......\    /                     ||
            ______ =       =|__ /__                     ||      \\\
        ,--'

In [18]:
obj.ArchR


           ___      .______        ______  __    __  .______      
          /   \     |   _  \      /      ||  |  |  | |   _  \     
         /  ^  \    |  |_)  |    |  ,----'|  |__|  | |  |_)  |    
        /  /_\  \   |      /     |  |     |   __   | |      /     
       /  _____  \  |  |\  \\___ |  `----.|  |  |  | |  |\  \\___.
      /__/     \__\ | _| `._____| \______||__|  |__| | _| `._____|
    



class: ArchRProject 
outputDirectory: /maps/projects/ralab_nnfc-AUDIT/people/lpm537/project/E2G/analysis/E2G_240503/data/Jurkat/1.Genome_wide_prediction/ArchR/ArchR/ArchR 
samples(1): Jurkat
sampleColData names(1): ArrowFiles
cellColData names(13): Sample TSSEnrichment ... nDiFrags BlacklistRatio
numberOfCells(1): 5276
medianTSS(1): 15.406
medianFrags(1): 9551

In [19]:
obj.ArchR = addIterativeLSI(
    ArchRProj = obj.ArchR,
    useMatrix = "TileMatrix", 
    name = "IterativeLSI"
)

Checking Inputs...

ArchR logging to : ArchRLogs/ArchR-addIterativeLSI-6788d5666bbbb-Date-2025-03-22_Time-16-59-36.00687.log
If there is an issue, please report to github with logFile!

2025-03-22 16:59:36.684237 : Computing Total Across All Features, 0 mins elapsed.

2025-03-22 16:59:37.522429 : Computing Top Features, 0.014 mins elapsed.

###########
2025-03-22 16:59:39.834307 : Running LSI (1 of 2) on Top Features, 0.053 mins elapsed.
###########

2025-03-22 16:59:39.849399 : Creating Partial Matrix, 0.053 mins elapsed.

2025-03-22 16:59:49.347251 : Computing LSI, 0.211 mins elapsed.

2025-03-22 17:00:38.598938 : Identifying Clusters, 1.032 mins elapsed.

Warning message:
“Data is of class matrix. Coercing to dgCMatrix.”
2025-03-22 17:00:44.199953 : Identified 6 Clusters, 1.126 mins elapsed.

2025-03-22 17:00:44.213906 : Saving LSI Iteration, 1.126 mins elapsed.

Found more than one class "dist" in cache; using the first, from namespace 'BiocGenerics'

Also defined by ‘spam’

Found 


************************************************************
2025-03-22 17:00:54.339123 : ERROR Found in .saveIteration for  
LogFile = ArchRLogs/ArchR-addIterativeLSI-6788d5666bbbb-Date-2025-03-22_Time-16-59-36.00687.log

<simpleError in g$grobs[[legend]]: no such index at level 2
>

************************************************************



2025-03-22 17:00:54.341443 : Creating Cluster Matrix on the total Group Features, 1.295 mins elapsed.

2025-03-22 17:01:06.56513 : Computing Variable Features, 1.498 mins elapsed.

###########
2025-03-22 17:01:06.751856 : Running LSI (2 of 2) on Variable Features, 1.501 mins elapsed.
###########

2025-03-22 17:01:06.767986 : Creating Partial Matrix, 1.502 mins elapsed.

2025-03-22 17:01:18.118703 : Computing LSI, 1.691 mins elapsed.

2025-03-22 17:02:07.274138 : Finished Running IterativeLSI, 2.51 mins elapsed.



In [20]:
obj.ArchR = addPeakSet(obj.ArchR,
                       peakSet = reduce(pairs.E2G))

In [21]:
obj.ArchR = addPeakMatrix(obj.ArchR)

ArchR logging to : ArchRLogs/ArchR-addPeakMatrix-6788d19ca54a3-Date-2025-03-22_Time-17-02-09.765385.log
If there is an issue, please report to github with logFile!

2025-03-22 17:02:10.240696 : Batch Execution w/ safelapply!, 0 mins elapsed.

2025-03-22 17:02:10.821989 : Adding Jurkat to PeakMatrix for Chr (1 of 23)!, 0.009 mins elapsed.

2025-03-22 17:02:17.822808 : Adding Jurkat to PeakMatrix for Chr (2 of 23)!, 0.126 mins elapsed.

2025-03-22 17:02:23.649179 : Adding Jurkat to PeakMatrix for Chr (3 of 23)!, 0.223 mins elapsed.

2025-03-22 17:02:29.03354 : Adding Jurkat to PeakMatrix for Chr (4 of 23)!, 0.313 mins elapsed.

2025-03-22 17:02:33.936394 : Adding Jurkat to PeakMatrix for Chr (5 of 23)!, 0.394 mins elapsed.

2025-03-22 17:02:38.993649 : Adding Jurkat to PeakMatrix for Chr (6 of 23)!, 0.479 mins elapsed.

2025-03-22 17:02:44.170674 : Adding Jurkat to PeakMatrix for Chr (7 of 23)!, 0.565 mins elapsed.

2025-03-22 17:02:49.339962 : Adding Jurkat to PeakMatrix for Chr (8 of 2

In [22]:
matrix.rna.rename2 = matrix.rna.rename
colnames(matrix.rna.rename2) = paste(celltype,colnames(matrix.rna.rename),sep="#")

In [23]:
seRNA <- SummarizedExperiment(
  assays = SimpleList(counts = matrix.rna.rename2),
  rowRanges = gr.tss
)

In [24]:
seRNA = sort.GenomicRanges(sortSeqlevels(seRNA), 
                           ignore.strand = TRUE)

In [25]:
seRNA

class: RangedSummarizedExperiment 
dim: 17571 5277 
metadata(0):
assays(1): counts
rownames(17571): OR4F5 FAM87B ... TTTY10 EIF1AY
rowData names(4): name score Ensembl_ID gene_type
colnames(5277): Jurkat#e10l1_AAACCAGGTACTTATG-1
  Jurkat#e10l1_AAACCAGGTCACACTG-1 ... Jurkat#e10l2_TTTGTGAGTCCTGAGC-1
  Jurkat#e10l2_TTTGTGAGTTTATGTG-1
colData names(0):

In [26]:
obj.ArchR = addGeneExpressionMatrix(obj.ArchR,
                                    seRNA = seRNA,
                                    strictMatch = T)

ArchR logging to : ArchRLogs/ArchR-addGeneExpressionMatrix-6788d10b221ad-Date-2025-03-22_Time-17-04-06.821096.log
If there is an issue, please report to github with logFile!

Overlap w/ scATAC = 1

2025-03-22 17:04:07.454319 : 

Overlap Per Sample w/ scATAC : Jurkat=5276

2025-03-22 17:04:07.463572 : 

2025-03-22 17:04:08.490168 : Batch Execution w/ safelapply!, 0 mins elapsed.

2025-03-22 17:04:09.420528 : Adding Jurkat to GeneExpressionMatrix for Chr (1 of 23)!, 0.016 mins elapsed.

2025-03-22 17:04:13.320135 : Adding Jurkat to GeneExpressionMatrix for Chr (2 of 23)!, 0.08 mins elapsed.

2025-03-22 17:04:16.997553 : Adding Jurkat to GeneExpressionMatrix for Chr (3 of 23)!, 0.142 mins elapsed.

2025-03-22 17:04:22.166369 : Adding Jurkat to GeneExpressionMatrix for Chr (4 of 23)!, 0.228 mins elapsed.

2025-03-22 17:04:25.761657 : Adding Jurkat to GeneExpressionMatrix for Chr (5 of 23)!, 0.288 mins elapsed.

2025-03-22 17:04:29.384099 : Adding Jurkat to GeneExpressionMatrix for Chr (6 o

Run ArchR prediction

In [27]:
start_time <- Sys.time()
obj.ArchR = addPeak2GeneLinks(
    ArchRProj = obj.ArchR,
    reducedDims = "IterativeLSI",
    useMatrix = "GeneExpressionMatrix",
    maxDist = maxDist,
    threads = n.cores
)
end_time <- Sys.time()
execution_time <- end_time - start_time

ArchR logging to : ArchRLogs/ArchR-addPeak2GeneLinks-6788d4d422b55-Date-2025-03-22_Time-17-05-46.82779.log
If there is an issue, please report to github with logFile!

2025-03-22 17:05:47.398093 : Getting Available Matrices, 0.009 mins elapsed.

No predictionScore found. Continuing without predictionScore!

2025-03-22 17:05:48.12356 : Filtered Low Prediction Score Cells (0 of 5276, 0), 0 mins elapsed.

2025-03-22 17:05:48.198036 : Computing KNN, 0.001 mins elapsed.

2025-03-22 17:05:48.430971 : Identifying Non-Overlapping KNN pairs, 0.005 mins elapsed.

2025-03-22 17:05:49.933534 : Identified 492 Groupings!, 0.03 mins elapsed.

2025-03-22 17:05:49.96273 : Getting Group RNA Matrix, 0.031 mins elapsed.

2025-03-22 17:06:02.290691 : Getting Group ATAC Matrix, 0.236 mins elapsed.

2025-03-22 17:06:28.564878 : Normalizing Group Matrices, 0.674 mins elapsed.

2025-03-22 17:06:33.81362 : Finding Peak Gene Pairings, 0.762 mins elapsed.

2025-03-22 17:06:34.551157 : Computing Correlations, 0.77

In [28]:
execution_time

Time difference of 1.024715 mins

In [29]:
p2g = metadata(obj.ArchR@peakSet)$Peak2GeneLinks

In [30]:
metadata(p2g)[[1]]$PeakName = paste(seqnames(metadata(p2g)[[1]]),
                                    ranges(metadata(p2g)[[1]]),sep="-")

In [31]:
p2g$PairName = paste(metadata(p2g)[[1]][p2g$idxATAC]$PeakName,
                     metadata(p2g)[[2]][p2g$idxRNA]$name,
                     sep = "_")

In [32]:
rownames(p2g) = p2g$PairName
p2g

DataFrame with 3125860 rows and 7 columns
                                idxATAC    idxRNA Correlation       FDR
                              <integer> <integer>   <numeric> <numeric>
chr1-10010-10640_OR4F5                1         1   0.0598673  0.399143
chr1-180669-181787_OR4F5              2         1   0.0298197  0.714636
chr1-191241-191629_OR4F5              3         1  -0.0701763  0.305854
chr1-267890-268113_OR4F5              4         1   0.0879553  0.176898
chr1-586090-586306_OR4F5              5         1  -0.0533506  0.463661
...                                 ...       ...         ...       ...
chrX-156001485-156001953_IL9R    152883     17554  0.06406394  0.359740
chrX-156008847-156009445_IL9R    152884     17554 -0.09324993  0.146873
chrX-156019836-156020186_IL9R    152885     17554 -0.04345845  0.567586
chrX-156029774-156030373_IL9R    152886     17554 -0.04818530  0.517289
chrX-156030497-156030826_IL9R    152887     17554  0.00467496  0.962017
                      

In [33]:
pairs.E2G.res = pairs.E2G[pairs.E2G$PairName %in% p2g$PairName]
mcols(pairs.E2G.res)[,c("Correlation","FDR","VarQATAC","VarQRNA")] = p2g[pairs.E2G.res$PairName,c("Correlation","FDR","VarQATAC","VarQRNA")]

In [34]:
pairs.E2G.res

GRanges object with 2590340 ranges and 7 metadata columns:
            seqnames            ranges strand |  TargetGene
               <Rle>         <IRanges>  <Rle> | <character>
        [1]     chr1 10001055-10001570      * | APITD1-CORT
        [2]     chr1 10001055-10001570      * |       CASZ1
        [3]     chr1 10001055-10001570      * |      CLSTN1
        [4]     chr1 10001055-10001570      * |    CTNNBIP1
        [5]     chr1 10001055-10001570      * |        DFFA
        ...      ...               ...    ... .         ...
  [2590336]     chrX   9941883-9942743      * |        WWC3
  [2590337]     chrX 99603024-99603776      * |      PCDH19
  [2590338]     chrX 99632013-99632219      * |      PCDH19
  [2590339]     chrX   9995829-9996207      * |       TBL1X
  [2590340]     chrX   9995829-9996207      * |        WWC3
                          PeakName               PairName Correlation
                       <character>            <character>   <numeric>
        [1] chr1-1000

Save results

In [35]:
saveRDS(obj.ArchR,
        paste(dir.output,"obj.ArchR.rds",sep = "/"))
saveRDS(pairs.E2G.res,
        paste(dir.output,"pairs.E2G.res.rds",sep = "/"))
df.output = as.data.frame(pairs.E2G.res,row.names = NULL)
colnames(df.output)[1] = "chr"
df.output[,"CellType"] = celltype
df.output = df.output[,c("chr",
                         "start",
                         "end",
                         "TargetGene",
                         "CellType",
                         "Correlation",
                         "FDR",
                         "VarQATAC",
                         "VarQRNA")]
data.table::fwrite(df.output,
                   file = paste(dir.output,"pairs.E2G.res.tsv.gz",sep = "/"),
                   row.names = F,
                   quote = F,
                   sep = "\t")
pairs.E2G.res
df.output

GRanges object with 2590340 ranges and 7 metadata columns:
            seqnames            ranges strand |  TargetGene
               <Rle>         <IRanges>  <Rle> | <character>
        [1]     chr1 10001055-10001570      * | APITD1-CORT
        [2]     chr1 10001055-10001570      * |       CASZ1
        [3]     chr1 10001055-10001570      * |      CLSTN1
        [4]     chr1 10001055-10001570      * |    CTNNBIP1
        [5]     chr1 10001055-10001570      * |        DFFA
        ...      ...               ...    ... .         ...
  [2590336]     chrX   9941883-9942743      * |        WWC3
  [2590337]     chrX 99603024-99603776      * |      PCDH19
  [2590338]     chrX 99632013-99632219      * |      PCDH19
  [2590339]     chrX   9995829-9996207      * |       TBL1X
  [2590340]     chrX   9995829-9996207      * |        WWC3
                          PeakName               PairName Correlation
                       <character>            <character>   <numeric>
        [1] chr1-1000

chr,start,end,TargetGene,CellType,Correlation,FDR,VarQATAC,VarQRNA
<fct>,<int>,<int>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
chr1,10001055,10001570,APITD1-CORT,Jurkat,-0.0654033111,0.347569890,0.2452792,0.31787627
chr1,10001055,10001570,CASZ1,Jurkat,0.0852744357,0.193500470,0.2452792,0.40526376
chr1,10001055,10001570,CLSTN1,Jurkat,0.1068869545,0.086523359,0.2452792,0.84727128
chr1,10001055,10001570,CTNNBIP1,Jurkat,0.1667059261,0.003349231,0.2452792,0.85564544
chr1,10001055,10001570,DFFA,Jurkat,-0.0537373593,0.459680288,0.2452792,0.65574798
chr1,10001055,10001570,GPR157,Jurkat,0.0249170647,0.766729796,0.2452792,0.45909764
chr1,10001055,10001570,H6PD,Jurkat,-0.0666460691,0.336484757,0.2452792,0.48484676
chr1,10001055,10001570,KIF1B,Jurkat,0.0204891917,0.812859646,0.2452792,0.85558847
chr1,10001055,10001570,LINC01759,Jurkat,0.0471842099,0.527837922,0.2452792,0.07855759


In [36]:
sessionInfo()

R version 4.3.3 (2024-02-29)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Red Hat Enterprise Linux 8.10 (Ootpa)

Matrix products: default
BLAS/LAPACK: /maps/projects/ralab/people/lpm537/software/anaconda3/envs/Notebook_E2G_240505/lib/libopenblasp-r0.3.27.so;  LAPACK version 3.12.0

Random number generation:
 RNG:     L'Ecuyer-CMRG 
 Normal:  Inversion 
 Sample:  Rejection 
 
locale:
 [1] LC_CTYPE=en_US.UTF-8       LC_NUMERIC=C              
 [3] LC_TIME=en_US.UTF-8        LC_COLLATE=en_US.UTF-8    
 [5] LC_MONETARY=en_US.UTF-8    LC_MESSAGES=en_US.UTF-8   
 [7] LC_PAPER=en_US.UTF-8       LC_NAME=C                 
 [9] LC_ADDRESS=C               LC_TELEPHONE=C            
[11] LC_MEASUREMENT=en_US.UTF-8 LC_IDENTIFICATION=C       

time zone: Europe/Copenhagen
tzcode source: system (glibc)

attached base packages:
 [1] parallel  stats4    grid      stats     graphics  grDevices utils    
 [8] datasets  methods   base     

other attached packages:
 [1] nabor_0.5.0           